<a href="https://colab.research.google.com/github/nekoniii3/fighter_news_bot/blob/main/bak/GONG%E3%81%8B%E3%82%89%E3%81%AE%E3%83%8B%E3%83%A5%E3%83%BC%E3%82%B9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Gongからのニュース一覧を取得します<br>**
https://gonkaku.jp/news?category=mma

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import datetime

# GONGからのニュースを取得（5件程度）
def gong_topic():

  URL = "https://gonkaku.jp/news?category=mma"

  DOMAIN = "https://gonkaku.jp"

  # 日付のフォーマット
  DT_PATTERN = r'\d{4}.\d{2}.\d{2}'

  # 今日の日本の時間を取得しておく
  t_delta = datetime.timedelta(hours=9)
  JST = datetime.timezone(t_delta, 'JST')
  dt_today = datetime.datetime.now(JST)

  # HTMLの取得先として当サイトを指定する
  response = requests.get(URL)

  # BeautifulSoup4でサイトのテキストを取得し、
  # 第二因数にhtml.parserを指定、解析結果をsoupに入れる
  response.encoding = response.apparent_encoding

  # print(response.text)
  soup = BeautifulSoup(response.text,'html.parser')

  # ニュースの欄を取得（ランキングを除くため）
  # ※広告で2つに分かれているが最初のブロックだけ取得
  news = soup.find("ul", class_="news-article-list")

  # 記事リスト取得 ※スペシャル除く（article-special-item category-mma）
  # elems_list = news.findAll(True, class_="article-news-item medium type-news", class_=re.compile(r"^l"))
  elems_list = news.findAll(True, class_=re.compile(r"article-.*-item.*"))

  # データ格納用DataFrame
  df = pd.DataFrame()

  for el in elems_list:

    div_date = el.find("div", class_=re.compile(r".*-date"))

    if re.fullmatch(DT_PATTERN, div_date.text):
      date = datetime.datetime.strptime(div_date.text, '%Y.%m.%d')
    else:
      date = dt_today.replace(hour=0, minute=0, second=0, microsecond=0,tzinfo=None)

    div_title = el.find("div", class_="article-title")
    title = div_title.text

    a = el.find("a")
    url = DOMAIN + a.get("href")

    df1 = pd.DataFrame([[date,"GONG",title,url]])

    df = pd.concat([df,df1])

  df = df.rename(columns={0:'Date',1: 'Source', 2: 'Title', 3: 'Url'})

  return df


In [ ]:
df_go = gong_topic()

# 一覧を表示
print(df_go)